In [1]:
import os 
from dotenv import load_dotenv
load_dotenv()
from langchain_groq import ChatGroq
groq_api = os.getenv("GROQ_API_KEY")

llm = ChatGroq(model="llama-3.3-70b-versatile")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000018AD6ED1940>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000018AD6ED2660>, model_name='llama-3.3-70b-versatile', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [10]:
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings
from langchain_classic.chains import create_retrieval_chain
from langchain_classic.chains.combine_documents import create_stuff_documents_chain

In [7]:
embedding = OllamaEmbeddings(model="embeddinggemma:300m")

In [15]:
from bs4 import SoupStrainer
loader = WebBaseLoader(web_path=("https://en.wikipedia.org/wiki/World_War_II"))
docs = loader.load()
print(docs)

[Document(metadata={'source': 'https://en.wikipedia.org/wiki/World_War_II', 'title': 'World War II - Wikipedia', 'language': 'en'}, page_content='\n\n\n\nWorld War II - Wikipedia\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nJump to content\n\n\n\n\n\n\n\nMain menu\n\n\n\n\n\nMain menu\nmove to sidebar\nhide\n\n\n\n\t\tNavigation\n\t\n\n\nMain pageContentsCurrent eventsRandom articleAbout WikipediaContact us\n\n\n\n\n\n\t\tContribute\n\t\n\n\nHelpLearn to editCommunity portalRecent changesUpload fileSpecial pages\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAppearance\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nDonate\n\nCreate account\n\nLog in\n\n\n\n\n\n\n\n\nPersonal tools\n\n\n\n\n\nDonate Create account Log in\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nContents\nmove to sidebar\nhide\n\n\n\n\n(Top)\n\n\n\n\n\n1\nStart and end dates\n\n\n\n\n\n\n\n\n2\nBackground\n\n\n\n\nToggle Background 

In [16]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500,chunk_overlap=300)
splits=text_splitter.split_documents(docs)
vector_store=Chroma.from_documents(documents=splits,embedding=embedding)
retriever=vector_store.as_retriever()

In [18]:
system_prompt = (
    "You are an assistant for question answering tasks. "
    "Use the following piece of retrieved context to answer the question. "
    "If you do not know the answer, say that you do not know. "
    "Use three sentences maximum. Keep the answer concise.\n{context}"
)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        ("human","{input}")
    ]
)

In [19]:
question_answer_chain=create_stuff_documents_chain(llm,prompt)
rag_chain = create_retrieval_chain(retriever,question_answer_chain)

In [25]:
from IPython.display import Markdown

In [32]:
response=rag_chain.invoke({"input":"how many soldiers were in the red army?"})
display(Markdown(response['answer']))

The total number of soldiers in the Red Army during World War II is not explicitly stated in the provided context. However, it is mentioned that the Soviet Union lost around 27 million people during the war, including 8.7 million military deaths.